## Yelp Dataset Pruning

### Purpose

This script (`yelp_preprocessing.py`) is designed to prune the Yelp dataset obtained online, making it suitable for use with scikit-learn machine learning models. The pruning process involves cleaning and refining the dataset to ensure it aligns with the requirements of scikit-learn.

### Steps

1. **Data Input:**
   - Ensure the Yelp dataset in JSON format is available online.

2. **Run Script:**
   - Execute `python yelp_preprocessing.py` to run the pruning script.

3. **Output:**
   - The script generates a pruned version of the Yelp dataset ready for scikit-learn usage.

### Notes

- Verify that the pruned features are in accordance with scikit-learn model requirements.
- Keep a backup of the original dataset obtained online.


#### Dependencies

In [38]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install altair

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 996 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 332 kB 46.0 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install rtree

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 398 kB 1.9 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.1 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 22.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 49.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 16.1 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install census

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install us

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 323 kB 2.7 MB/s eta 0:00:01
  Created wheel for us: filename=us-3.1.1-py3-none-any.whl size=12554 sha256=12fb9bd9c5b8ee4765345eb85cbf9fa3e2e2655206706dfbdd4b0322c444fc01
  Stored in directory: /Users/mihir/Library/Caches/pip/wheels/29/9e/92/3672525fc19ac574d668402d739c8e8ff4650012304d2f3f06
Successfully built us
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [59]:
#Import Relevant Packages
import pandas as pd
import numpy as np
import altair as alt
from census import Census
from us import states

In [7]:
#Import CSV File
df = pd.read_csv("./yelp_dataset.csv", #Directory
                 usecols=['business_id',
                          'name',
                          'address',
                          'city',
                          'state',
                          'latitude',
                          'longitude',
                          'stars',
                          'review_count',
                          'categories'])

In [8]:
#Shape
print(df.shape)
print(df.head())

(150346, 10)
              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state   latitude  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA  34.426679   
1  87 Grasso Plaza Shopping Center         Affton    MO  38.551126   
2             5255 E Broadway Blvd         Tucson    AZ  32.223236   
3                      935 Race St   Philadelphia    PA  39.955505   
4                    101 Walnut St     Green Lane    PA  40.338183   

    longitude  stars  review_count  \
0 -119.711197    5.0             7   
1  -90.335695    3.0            15   
2 -110.880452    3.5            22   
3  -75.155564    4.0            80   
4  -75.471659    4.5            13   

      

In [9]:
#Filters
df1 = df[df['categories'].str.contains('restaurants', case=False, na=False)]
print(df1.shape)
print(df1.head())

(52268, 10)
               business_id                   name              address  \
3   MTSW4McQd7CbVtyjqoe9mw     St Honore Pastries          935 Race St   
5   CF33F8-E6oudUQ46HnavjQ         Sonic Drive-In        615 S Main St   
8   k0hlBqXX-Bt0vf1op7Jr1w  Tsevi's Pub And Grill    8025 Mackenzie Rd   
9   bBDDEgkFA1Otx9Lfe7BZUQ         Sonic Drive-In  2312 Dickerson Pike   
11  eEOYSgkmpB90uNA7lDOMRA  Vietnamese Food Truck                  NaN   

            city state   latitude  longitude  stars  review_count  \
3   Philadelphia    PA  39.955505 -75.155564    4.0            80   
5   Ashland City    TN  36.269593 -87.058943    2.0             6   
8         Affton    MO  38.565165 -90.321087    3.0            19   
9      Nashville    TN  36.208102 -86.768170    1.5            10   
11     Tampa Bay    FL  27.955269 -82.456320    4.0            10   

                                           categories  
3   Restaurants, Food, Bubble Tea, Coffee & Tea, B...  
5   Burgers, Fas

In [11]:
df2 = df1.iloc[:, [5, 6, 7, 8, 9]] #Remove address columns and name columns
print(df2.shape)
print(df2.head())

(52268, 5)
     latitude  longitude  stars  review_count  \
3   39.955505 -75.155564    4.0            80   
5   36.269593 -87.058943    2.0             6   
8   38.565165 -90.321087    3.0            19   
9   36.208102 -86.768170    1.5            10   
11  27.955269 -82.456320    4.0            10   

                                           categories  
3   Restaurants, Food, Bubble Tea, Coffee & Tea, B...  
5   Burgers, Fast Food, Sandwiches, Food, Ice Crea...  
8   Pubs, Restaurants, Italian, Bars, American (Tr...  
9   Ice Cream & Frozen Yogurt, Fast Food, Burgers,...  
11         Vietnamese, Food, Restaurants, Food Trucks  


### Census Dataset from CenPy

In [20]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.5 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 8.1 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [95]:
c = Census("4ec0e19095dc0c441cd14bf28bc3f943cb8f2bb7")

In [101]:
#Chao Code Block
c.acs5.state(('NAME', 'B25034_010E'), states.MD.fips)

APIKeyError: ' <html>     <head>         <title>Invalid Key</title>     </head>     <body>         <p>             A valid <em>key</em> must be included with each data API request.             You included a key with this request, however, it is not valid.             Please check your key and try again.         </p>         <p>             If you do not have a key you may sign up for one <a href="key_signup.html">here</a>.         </p>     </body> </html>'

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity